# ETL de Gastos Pessoais

Projeto desenvolvido como parte do Santander Bootcamp 2025, com foco na implementação de um pipeline ETL (Extração, Transformação e Carregamento) utilizando Python.

# Import Required Libraries

In [2]:
import os
import pandas as pd
import pathlib
import sys

# Extract

In [3]:
def extract():
    csv_path = pathlib.Path("data/gastos.csv")

    try:

        # Leitura do arquivo CSV
        df = pd.read_csv(csv_path)
        print("Extração concluída com sucesso!")
        print(f"Total de registros carregados: {len(df)}")

        return df

    except FileNotFoundError:
        print("Erro: arquivo gastos.csv não encontrado.")
        sys.exit(1)

    except Exception as erro:
        print(f"Erro inesperado durante a extração: {erro}")
        sys.exit(1)

df = extract()
df.head()

Extração concluída com sucesso!
Total de registros carregados: 300


,data,categoria,valor,descricao
0,2025-04-23,Educação,319.99,Curso online
1,2025-06-08,Transporte,205.00,Ônibus
2,2025-02-24,Moradia,356.48,Aluguel
3,2025-01-29,Saúde,239.67,Consulta médica
4,2025-06-07,Lazer,317.87,Viagem


# Transform

In [4]:
def transform(df):

    # Criar cópia para evitar efeitos colaterais
    df = df.copy()

    # Garantir que a coluna 'valor' seja numérica
    df["valor"] = pd.to_numeric(df["valor"], errors="coerce")

    # Remover linhas inválidas
    df = df.dropna(subset=["valor"])

    # Agrupar os dados por categoria e somar os valores
    resumo = (
        df
        .groupby("categoria", as_index=False)["valor"]
        .sum()
    )

    # Renomear coluna
    resumo.rename(columns={"valor": "total_gasto"}, inplace=True)

    # Ordenar do maior para o menor
    resumo = resumo.sort_values(by="total_gasto", ascending=False)

    # Resetar índice
    resumo = resumo.reset_index(drop=True)

    # Categoria com maior gasto (primeira linha já ordenada)
    categoria_maior_gasto = resumo.loc[0, "categoria"]

    print(f"Categoria com maior gasto: {categoria_maior_gasto}")

    return resumo


resumo = transform(df)
resumo

Categoria com maior gasto: Moradia


,categoria,total_gasto
0,Moradia,18287.74
1,Saúde,17796.25
2,Lazer,15416.93
3,Alimentação,14288.37
4,Educação,13779.66
5,Transporte,10513.63


# Load

In [5]:
def load(resumo):

    # Definir caminho do arquivo de saída
    pasta_saida = "data"
    arquivo_saida = "resumo_gastos.csv"
    caminho_saida = os.path.join(pasta_saida, arquivo_saida)

    # Garantir que a pasta de saída exista
    os.makedirs(pasta_saida, exist_ok=True)

    # Salvar o DataFrame resumo em um arquivo CSV
    resumo.to_csv(caminho_saida, index=False)

    print("Carregamento concluído com sucesso!")
    print(f"Arquivo salvo em: {caminho_saida}")

load(resumo)

Carregamento concluído com sucesso!
Arquivo salvo em: data\resumo_gastos.csv
